#  Load Data

In [7]:
import pandas as pd

def load_data(csv_file_path):
    df = pd.read_csv(csv_file_path)

    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    print("Data overview:")
    print(df.head())
    print("\nClass distribution:")
    print(df['Human Status'].value_counts())

    return df

# Run this part first
csv_file_path = "https://raw.githubusercontent.com/ishancoderr/WiSee/refs/heads/main/data/finalDataset.csv"
df = load_data(csv_file_path)

Data overview:
   Tile No  Receiver 1  Receiver 2  Receiver 3  Human Status
0        1         -37         -42         -34             0
1        1         -37         -42         -32             0
2        1         -37         -42         -32             0
3        1         -36         -41         -34             0
4        1         -39         -42         -35             0

Class distribution:
Human Status
2    818
1    734
0    629
Name: count, dtype: int64


# Prepare Data

In [8]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
import joblib

def prepare_data(df):
    # Extract features and target
    X = df[['Receiver 1', 'Receiver 2', 'Receiver 3']].values
    y = df['Human Status'].values

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    joblib.dump(scaler, 'scaler.save')

    X_train_cnn = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
    X_test_cnn = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

    # One-hot encode targets (e.g., 0 -> [1, 0, 0])
    y_train_categorical = to_categorical(y_train)
    y_test_categorical = to_categorical(y_test)

    return X_train_cnn, X_test_cnn, y_train_categorical, y_test_categorical, y_train, y_test


X_train, X_test, y_train_cat, y_test_cat, y_train, y_test = prepare_data(df)

# Build the CNN Model

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

def create_cnn_model(num_classes):
    model = Sequential()

    model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(3, 1)))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Conv1D(filters=32, kernel_size=1, activation='relu'))

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

num_classes = len(np.unique(y_train))
model = create_cnn_model(num_classes)
model.summary()

d:\WiSee\myenv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_10 (Conv1D)              │ (None, 2, 64)          │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 2, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_11 (Conv1D)              │ (None, 2, 32)          │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,627 (25.89 KB)

 Trainable params: 6,627 (25.89 KB)

 Non-trainable params: 0 (0.00 B)

# Train the Model

In [10]:
from tensorflow.keras.callbacks import EarlyStopping

def train_model(model, X_train, y_train, X_test, y_test, epochs=100, batch_size=32):

    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )

    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_test, y_test),
        callbacks=[early_stopping],
        verbose=1
    )

    return model, history

trained_model, history = train_model(model, X_train, y_train_cat, X_test, y_test_cat)

Epoch 1/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.6855 - loss: 0.8883 - val_accuracy: 0.8810 - val_loss: 0.3703
Epoch 2/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8818 - loss: 0.3544 - val_accuracy: 0.9062 - val_loss: 0.2456
Epoch 3/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9035 - loss: 0.2619 - val_accuracy: 0.9039 - val_loss: 0.2168
Epoch 4/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9203 - loss: 0.2269 - val_accuracy: 0.9130 - val_loss: 0.1954
Epoch 5/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9193 - loss: 0.2084 - val_accuracy: 0.9314 - val_loss: 0.1804
Epoch 6/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9261 - loss: 0.1745 - val_accuracy: 0.9268 - val_loss: 0.1658
Epoch 7/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9423 - loss: 0.1608 - val_accuracy: 0.9336 - val_loss: 0.1623
Epoch 8/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9421 - loss: 0.1582 - val_accuracy: 0.9428 -

# Evaluate the Model

In [11]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def evaluate_model(model, X_test, y_test_categorical, y_test, history):
    test_loss, test_accuracy = model.evaluate(X_test, y_test_categorical)
    print(f"Test accuracy: {test_accuracy:.4f}")

    y_pred_proba = model.predict(X_test)
    y_pred = np.argmax(y_pred_proba, axis=1)
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    model.save('rssi_human_detection_model.h5')
    print("Model saved as 'rssi_human_detection_model.h5'")

    return y_pred

y_pred = evaluate_model(trained_model, X_test, y_test_cat, y_test, history)


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9794 - loss: 0.0862 
Test accuracy: 0.9794
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step



Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       126
           1       0.96      0.98      0.97       147
           2       0.99      0.97      0.98       164

    accuracy                           0.98       437
   macro avg       0.98      0.98      0.98       437
weighted avg       0.98      0.98      0.98       437

Model saved as 'rssi_human_detection_model.h5'
